<a href="https://colab.research.google.com/github/Mercury-SL/Eficiencia-no-Sistema-Portuario/blob/main/AtracacaoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#bibliotecas iniciais
!pip install pandas
import io

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from google.colab import drive
drive.mount('/content/drive')

path_in = '/content/drive/My Drive/quarto semstre/Data/'
path_out = '/content/drive/My Drive/quarto semstre/Data/'

file1 = path_in + '2014Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file2 = path_in + '2015Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file3= path_in + '2016Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file4 = path_in + '2017Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file5 = path_in + '2018Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file6 = path_in + '2019Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file7 = path_in + '2020Atracacao.csv'
df1 = pd.read_csv(file1, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file8 = path_in + '2021Atracacao.csv'
df2 = pd.read_csv(file2, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file9 = path_in + '2022Atracacao.csv'
df3 = pd.read_csv(file3, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

file10 = path_in + '2023Atracacao.csv'
df4 = pd.read_csv(file4, low_memory=False, sep=';', encoding = 'UTF-8', decimal =",")

csv_final = [file1, file2, file3, file4, file5, file6, file7, file8, file9, file10]
ANTAQ_Int = pd.DataFrame()
for csv_final in csv_final:
    df = pd.read_csv(csv_final, sep = ';')
    ANTAQ_Int = pd.concat([ANTAQ_Int, df], ignore_index=True)
ANTAQ_Int.info()

ANTAQ_Int.head(5)

colunas_interesse = ['IDBerco','Data Atracação', 'Data Chegada',
                     'Data Desatracação', 'Data Início Operação',
                     'Data Término Operação']
ANTAQ_Limpa = ANTAQ_Int.dropna(subset=colunas_interesse)
ANTAQ_Limpa.info()

ANTAQ_Limpa.head(5)

ANTAQ_Limpa['Data Chegada'] = pd.to_datetime(ANTAQ_Limpa['Data Chegada'], format='%d/%m/%Y %H:%M')
ANTAQ_Limpa['Data Atracação'] = pd.to_datetime(ANTAQ_Limpa['Data Atracação'], format='%d/%m/%Y %H:%M')
ANTAQ_Limpa['Data Início Operação'] = pd.to_datetime(ANTAQ_Limpa['Data Início Operação'], format='%d/%m/%Y %H:%M')
ANTAQ_Limpa['Data Término Operação'] = pd.to_datetime(ANTAQ_Limpa['Data Término Operação'], format='%d/%m/%Y %H:%M')
ANTAQ_Limpa['Data Desatracação'] = pd.to_datetime(ANTAQ_Limpa['Data Desatracação'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

ANTAQ_Limpa['Tempo_espera_chegada'] = (ANTAQ_Limpa['Data Atracação'] - ANTAQ_Limpa['Data Chegada']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_espera_inicial'] = (ANTAQ_Limpa['Data Início Operação'] - ANTAQ_Limpa['Data Atracação']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_operacao'] = (ANTAQ_Limpa['Data Término Operação'] - ANTAQ_Limpa['Data Início Operação']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_espera_final'] = (ANTAQ_Limpa['Data Desatracação'] - ANTAQ_Limpa['Data Término Operação']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_ocupacao'] = (ANTAQ_Limpa['Data Desatracação'] - ANTAQ_Limpa['Data Atracação']).dt.total_seconds() / 3600

duplicados = ANTAQ_Limpa.duplicated(subset=['IDAtracacao'], keep=False)
if duplicados.any():
    print("Existem valores duplicados.")
    print(ANTAQ_Limpa[duplicados])
else:
    print("Coluna com valores unicos.")

coluna = 'Complexo Portuário'
CP = ANTAQ_Limpa[coluna].value_counts().sort_index()
print(f"Total de operações {coluna}:")
print(CP)

#define o filtro de CDTUP para casar com a base de cargas de interesse
porto = 'Santos'
navegacao = 'Longo Curso'
filtro =  ANTAQ_Limpa[(ANTAQ_Limpa['Complexo Portuário'] == porto) &
                (ANTAQ_Limpa['Tipo de Navegação da Atracação'] == navegacao)]
tabela = filtro.groupby('Berço').agg({'Berço': 'count'}).rename(columns={'Berço': 'Total'})
print(tabela)

filtro.info()

colunas_interesse1 = ['IDAtracacao','CDTUP','Berço','Porto Atracação', 'Complexo Portuário','Tipo de Navegação da Atracação',
                      'Data Chegada','Data Atracação','Data Início Operação','Data Término Operação','Data Desatracação',
                      'Tempo_espera_chegada', 'Tempo_espera_inicial','Tempo_operacao', 'Tempo_espera_final', 'Tempo_ocupacao']
filtro[colunas_interesse1].to_csv(path_out + 'Atracacao_final.csv', index=False)

drive.mount('/content/drive')
arquivo = '/content/drive/My Drive/Data/Atracacao_final.csv'
with open(arquivo, 'r', encoding='UTF-8') as f:
    linhas = f.readlines()
    for i in range(0, 5):
        print(f"Linha {i+1}: {linhas[i]}")

df = pd.read_csv(arquivo, low_memory=False, sep=',', encoding='UTF-8')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358279 entries, 0 to 358278
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   IDAtracacao                     358279 non-null  int64  
 1   CDTUP                           358279 non-null  object 
 2   IDBerco                         356226 non-null  object 
 3   Berço                           356143 non-null  object 
 4   Porto Atracação                 356226 non-null  object 
 5   Coordenadas                     356090 non-null  object 
 6   Apelido Instalação Portuária    100263 non-null  object 
 7   Complexo Portuário              358279 non-null  object 
 8   Tipo da Autoridade Portuária    358279 non-null  object 
 9   Data Atracação                  356226 non-null  object 
 10  Data Chegad

<ipython-input-6-d7f82483dea2>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ANTAQ_Limpa['Data Chegada'] = pd.to_datetime(ANTAQ_Limpa['Data Chegada'], format='%d/%m/%Y %H:%M')
<ipython-input-6-d7f82483dea2>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ANTAQ_Limpa['Data Atracação'] = pd.to_datetime(ANTAQ_Limpa['Data Atracação'], format='%d/%m/%Y %H:%M')
<ipython-input-6-d7f82483dea2>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

Coluna com valores unicos.
Total de operações Complexo Portuário:
Complexo Portuário
Almeirim                                    2
Angra dos Reis                             57
Aracajú                                   413
Aratu - Salvador                         3294
Areia Branca                             8307
Areia Branca (Instalação Isolada)        9048
Barra do Riacho                            23
Cabedelo (Instalação Isolada)             540
Coari (Instalação Isolada)                  1
Corumbá                                 22127
Estrela                                     9
Forno                                     434
Ilhéus                                    364
Imbituba                                 1498
Itaguaí                                  6518
Itaituba                                15783
Itajaí                                    293
Itaqui                                  13649
LDC Pederneiras (Instalação Isolada)     2389
Macapá                                   

,IDAtracacao,CDTUP,Berço,Porto Atracação,Complexo Portuário,Tipo de Navegação da Atracação,Data Chegada,Data Atracação,Data Início Operação,Data Término Operação,Data Desatracação,Tempo_espera_chegada,Tempo_espera_inicial,Tempo_operacao,Tempo_espera_final,Tempo_ocupacao
0,579549,BRSP001,TUF1,Terminal Integrador Portuário Luiz Antonio Mes...,Santos,Longo Curso,2013-12-31 13:00:00,2013-12-31 15:40:00,2013-12-31 17:05:00,2014-01-03 10:20:00,NaN,2.666667,1.416667,65.250000,NaN,NaN
1,584408,BRSP002,Cais II #5,Terminal Marítimo Privativo de Cubatão - TMPC,Santos,Longo Curso,2014-01-01 13:05:00,2014-01-01 16:25:00,2014-01-01 17:30:00,2014-01-04 02:25:00,NaN,3.333333,1.083333,56.916667,NaN,NaN
2,584410,BRSP002,Cais II #5,Terminal Marítimo Privativo de Cubatão - TMPC,Santos,Longo Curso,2014-01-08 14:45:00,2014-01-08 19:00:00,2014-01-08 19:40:00,2014-01-10 04:20:00,NaN,4.250000,0.666667,32.666667,NaN,NaN
3,585300,BRSP002,Cais II #5,Terminal Marítimo Privativo de Cubatão - TMPC,Santos,Longo Curso,2014-01-24 00:10:00,2014-01-24 23:35:00,2014-01-25 01:05:00,2014-01-27 19:10:00,NaN,23.416667,1.500000,66.083333,NaN,NaN
4,585349,BRSP001,TUF1,Terminal Integrador Portuário Luiz Antonio Mes...,Santos,Longo Curso,2013-12-24 11:48:00,2014-01-03 17:30:00,2014-01-04 01:30:00,2014-01-09 16:30:00,NaN,245.700000,8.000000,135.000000,NaN,NaN
